# Initial Metrics of the Data

In [78]:
# import basic libraries
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [79]:
# loading data

ratings_data = pd.read_csv('ratings_small.csv.zip')
metadata = pd.read_csv('movies_metadata.csv.zip')
links_data = pd.read_csv('links.csv')

/tmp/ipykernel_36401/3465485731.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv('movies_metadata.csv.zip')


The columns obtained for each data frame are displayed below.

In [80]:
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [15]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [4]:
links_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


We can observe that metadata provides lots of information about each movie, giving us a lot of freedom when ploting and visualizing the data. Firstly, we should preprocess the data and do the corresponding mappings within the 3 dataframes.

Notice the following mappings and column relationships:

- The **movieId** column in *links_data* maps to the **movieId** column in *ratings_data*.

- The **imdbId** column in *links_data* maps to the **imdb_id** column in *metadata*.

#### Preprocessing the Metadata
In this next section, we preprocessed the data using the steps listed below. 

1. Removed rows from the metadata data frame where the imdb_id was null.
2. Convert each element of the imdb_id column in metadata to an int by applying a lambda function.
3. Merge metadata and links_data by joining the data frames on the imbd_id and imdbId columns respectively.


In [81]:
ratings_data['userId'] = ratings_data['userId'].astype('int32')

In [82]:
metadata = metadata[metadata['imdb_id'].notna()]

def remove_characters(string):
    
    return ''.join(filter(str.isdigit, string))
metadata['imdb_id'] = metadata['imdb_id'].apply(lambda x: int(remove_characters(str(x))))

In [83]:
full_metadata = pd.merge(metadata, links_data, left_on='imdb_id', right_on='imdbId')
full_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45383 entries, 0 to 45382
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45383 non-null  object 
 1   belongs_to_collection  4486 non-null   object 
 2   budget                 45383 non-null  object 
 3   genres                 45383 non-null  object 
 4   homepage               7765 non-null   object 
 5   id                     45383 non-null  object 
 6   imdb_id                45383 non-null  int64  
 7   original_language      45372 non-null  object 
 8   original_title         45383 non-null  object 
 9   overview               44433 non-null  object 
 10  popularity             45380 non-null  object 
 11  poster_path            45005 non-null  object 
 12  production_companies   45380 non-null  object 
 13  production_countries   45380 non-null  object 
 14  release_date           45302 non-null  object 
 15  re

'full_metadata' is a dataframe that can be used to retrieve the metadata for a movie based on the movie ID alone.

#### Creating a Spotlight Interactions Dataset

Spotlight has a dataset object that we need to use in order to train models on our data, called 'Interactions'. Below, we create an Interactions object by supplying the follownig parameters, all of wich must be Numpy arrays:

- **user_ids** - the user IDs in the rating data
- **item_ids** - the item IDs in the rating data
- **ratings** - the corresponding ratings in the rating data
- **timestamps (optional)** - the timestamps for each user/item interaction



In [84]:
from spotlight.interactions import Interactions

dataset = Interactions(user_ids= ratings_data['userId'].values,
                       item_ids= ratings_data['movieId'].values,
                       ratings = ratings_data['rating'].values,
                       timestamps= ratings_data['timestamp'].values)

#### Initial visualization of the data

In [14]:
full_metadata

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709,862.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497,8844.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,3,113228,15602.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,4,114885,31357.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,5,113041,11862.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45378,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,176269,6209470,439050.0
45379,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0,176271,2028550,111109.0
45380,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0,176273,303758,67758.0
45381,False,NaN,0,[],NaN,227506,8536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,[],Released,NaN,Satan Triumphant,False,0.0,0.0,176275,8536,227506.0


In [18]:
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int32  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int32(1), int64(2)
memory usage: 2.7 MB


In [19]:
full_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45383 entries, 0 to 45382
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45383 non-null  object 
 1   belongs_to_collection  4486 non-null   object 
 2   budget                 45383 non-null  object 
 3   genres                 45383 non-null  object 
 4   homepage               7765 non-null   object 
 5   id                     45383 non-null  object 
 6   imdb_id                45383 non-null  int64  
 7   original_language      45372 non-null  object 
 8   original_title         45383 non-null  object 
 9   overview               44433 non-null  object 
 10  popularity             45380 non-null  object 
 11  poster_path            45005 non-null  object 
 12  production_companies   45380 non-null  object 
 13  production_countries   45380 non-null  object 
 14  release_date           45302 non-null  object 
 15  re

In [85]:
full_metadata.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'movieId', 'imdbId', 'tmdbId'],
      dtype='object')

In [ ]:
# ratings and production companies

# movies and their popularity

# ratings and budget
# production companies and budget

#ratings and genres

Let's see what is the best and worse rated movie by the general public

In [86]:
# merge metadata and ratings

meta_ratings = full_metadata.merge(ratings_data, left_on='movieId', right_on='movieId')
meta_ratings

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,movieId,imdbId,tmdbId,userId,rating,timestamp
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1,114709,862.0,7,3.0,851866703
1,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1,114709,862.0,9,4.0,938629179
2,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1,114709,862.0,13,5.0,1331380058
3,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1,114709,862.0,15,2.0,997938310
4,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,1,114709,862.0,19,3.0,855190091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99773,False,"{'id': 286023, 'name': 'Sharknado Collection',...",0,"[{'id': 35, 'name': 'Comedy'}, {'id': 27, 'nam...",http://www.syfy.com/sharknado4,390989,4831420,en,Sharknado 4: The 4th Awakens,The new installment of the Sharknado franchise...,...,Sharknado 4: The 4th Awakens,False,4.3,88.0,161918,4831420,390989.0,624,1.5,1472929873
99774,False,NaN,8000000,"[{'id': 18, 'name': 'Drama'}]",NaN,159550,255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,The Last Brickmaker in America,False,7.0,1.0,161944,255313,159550.0,287,5.0,1470167824
99775,False,NaN,1000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 10749,...",NaN,392572,5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,Rustom,False,7.3,25.0,162542,5165344,392572.0,611,5.0,1471520667
99776,False,NaN,15050000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,402672,3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,Mohenjo Daro,False,6.7,26.0,162672,3859980,402672.0,611,3.0,1471523986


In [28]:
meta_ratings = meta_ratings[['rating', 'title']]
print(meta_ratings)

       rating                                              title
0         3.0                                          Toy Story
1         4.0                                          Toy Story
2         5.0                                          Toy Story
3         2.0                                          Toy Story
4         3.0                                          Toy Story
...       ...                                                ...
99773     1.5                       Sharknado 4: The 4th Awakens
99774     5.0                     The Last Brickmaker in America
99775     5.0                                             Rustom
99776     3.0                                       Mohenjo Daro
99777     5.0  The Beatles: Eight Days a Week - The Touring Y...

[99778 rows x 2 columns]


In [49]:
# Group by title and calculate the mean rating for each title
mean_ratings = meta_ratings.groupby('title')['rating'].mean().reset_index()
mean_ratings['rating'] = mean_ratings['rating']
mean_ratings

,title,rating
0,$9.99,3.833333
1,'Neath the Arizona Skies,0.500000
2,"'night, Mother",5.000000
3,(500) Days of Summer,3.755556
4,...And God Created Woman,5.000000
...,...,...
8733,À Nous la Liberté,4.500000
8734,Æon Flux,2.538462
8735,İtirazım Var,3.500000
8736,Želary,5.000000


In [51]:
# Sort the DataFrame by mean rating (optional but recommended)
mean_ratings = mean_ratings.sort_values(by='rating')
mean_ratings

,title,rating
6018,Superbabies: Baby Geniuses 2,0.5
3937,Malibu's Most Wanted,0.5
6734,The First Day of the Rest of Your Life,0.5
3906,Madea Goes to Jail,0.5
4816,Peter & the Wolf,0.5
...,...,...
719,Bang Bang You're Dead,5.0
6664,The Edge of Heaven,5.0
6661,The Earrings of Madame de...,5.0
6676,The End,5.0


In [60]:
print('There are',len(mean_ratings[mean_ratings['rating'] == 5]), 'movies rated in mean score of top value 5.')

print('There are ', len(mean_ratings[mean_ratings['rating'] <= 0.5]), 'movies rated with a mean score less than 0.5')

There are 300 movies rated in mean score of top value 5.
There are  89 movies rated with a mean score less than 0.5


Let's see metrics based on production companies

In [76]:
meta_ratings = meta_ratings[['rating', 'title', 'production_companies']]
print(meta_ratings)
# Group by title and calculate the mean rating for each title
mean_ratings = meta_ratings.groupby('title')['rating'].mean().reset_index()
mean_ratings = pd.merge(mean_ratings, meta_ratings[['title', 'production_companies']], on= 'title')

       rating                                              title  \
0         3.0                                          Toy Story   
1         4.0                                          Toy Story   
2         5.0                                          Toy Story   
3         2.0                                          Toy Story   
4         3.0                                          Toy Story   
...       ...                                                ...   
99773     1.5                       Sharknado 4: The 4th Awakens   
99774     5.0                     The Last Brickmaker in America   
99775     5.0                                             Rustom   
99776     3.0                                       Mohenjo Daro   
99777     5.0  The Beatles: Eight Days a Week - The Touring Y...   

                                    production_companies  
0         [{'name': 'Pixar Animation Studios', 'id': 3}]  
1         [{'name': 'Pixar Animation Studios', 'id': 3}]  
2     

In [77]:
import ast

mean_ratings['production_companies'] = mean_ratings['production_companies'].apply(ast.literal_eval)
print(mean_ratings)

mean_ratings['production_companies_name'] = mean_ratings['production_companies'].apply(lambda x: x['name'])

print(mean_ratings)

                          title    rating  \
0                         $9.99  3.833333   
1                         $9.99  3.833333   
2                         $9.99  3.833333   
3      'Neath the Arizona Skies  0.500000   
4                'night, Mother  5.000000   
...                         ...       ...   
99773                  Æon Flux  2.538462   
99774              İtirazım Var  3.500000   
99775                    Želary  5.000000   
99776           ’Round Midnight  2.250000   
99777           ’Round Midnight  2.250000   

                                    production_companies  
0      [{'name': 'Australian Film Finance Corporation...  
1      [{'name': 'Australian Film Finance Corporation...  
2      [{'name': 'Australian Film Finance Corporation...  
3      [{'name': 'Lone Star Production', 'id': 4114},...  
4                                                     []  
...                                                  ...  
99773  [{'name': 'Paramount Pictures', 'id': 4

IndexError: list index out of range